## Imports

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import emoji
from langdetect import detect

import re
import nltk
from num2words import num2words
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

import pymorphy2
from tqdm import tqdm
from autocorrect import Speller

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zlata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zlata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [95]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 163.8 kB/s eta 0:12:43
   ---------------------------------------- 0.0/124.9 MB 163.8 kB/s eta 0:12:43
   ---------------------------------------- 0.0/124.9 MB 163.4 kB/s eta 0:12:45
   ---------------------------------------- 0.1/124.9 MB 269.5 kB/s eta 0:07:44
   ---------------------------------------- 0.1/124.9 MB 327.2 kB/s eta 0:06:22
   ---------------------------------------- 0.2/124.9 MB 455.1 kB/s eta 0:04:35
   ---------------------------------------- 0.2/124.9 MB 602.4 kB/s eta 0:03:27
   ---------------------------------------- 0.3/124.9 MB 676.7 kB/s eta 0:03:05
   --


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [96]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

## Functions

In [27]:
TOKEN_RE = re.compile(r'[а-яё]+')
russian_stopwords = stopwords.words("russian")
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)
pymorphy2_311_hotfix()
lemmatizer = pymorphy2.MorphAnalyzer()
# corrector_fred = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_fredt5_large.value)
spell = Speller('ru')


def filt(value):
    if value in ('1', '1.0', 1):
        return 1
    elif value in ('0', '0.0', 0, 'o', "о"):
        return 0
    else:
        return -1
    
def drop_by_col(df, col):
    tmp = df[df[col].isna()].index
    df = df.drop(tmp)
    return df

def tokenize_text(txt, min_lenght_token=2):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_lenght_token]

def remove_special_chars(text):
    """Удаляет специальные символы из строки на русском языке"""
    # Регулярное выражение для поиска специальных символов
    pattern = r'[^\w\s\u0400-\u04FF\d]'
    return re.sub(pattern, '', text)

# Функция для преобразования эмоджи в слова
def emojis_words(text):
    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "), language='ru')
    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")   
    return clean_text

def convert_nums2words(text):
    words = []
    for word in text.split():
        if word.isdigit():
            words += [num2words(word, lang='ru')]
        else:
            words.append(word)
    return " ".join(words)

def remove_some(s, lst = ['игра', 'это', 'всё', 'который', 'весь']):
    for i in lst:
        s = s.replace(i, '')
    return s

def drop_trash(df):
    df = df.dropna()
    empty = df[(df.text == '') | (df.text == ' ')]
    df = df.drop(empty.index)
    df = df.drop(df[df.text.str.split().apply(len) == 1].text.index)
    df.text = df.text.apply(remove_some)
    df = df.drop_duplicates(subset='text', keep="last")
    return df

# Функция для очистки текста
def cleaning_text(input_text):    
    
    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', input_text)
    
    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    # Важно понимать эмоциональную окраску обрабатываемого текста
    clean_text = emojis_words(clean_text)
    
    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    # # Убираем специальные символы: избавляемся от всего, что не является "словами"
    # clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)

    # clean_text = corrector_fred.correct(clean_text)

    clean_text = spell(clean_text)

    clean_text = convert_nums2words(clean_text)

    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    tokens = tokenize_text(clean_text)
    lemmas = [lemmatizer.parse(token)[0].normal_form for token in tokens]
    lemmas = [lemma for lemma in lemmas if lemma not in stop_words]

    clean_text = ' '.join(lemmas)

    # # Знаки препинания: далее - удаляем из текста все знаки препинания
    # clean_text = re.sub(r'[^\w\s]', '', clean_text)

    clean_text = remove_special_chars(clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text



In [30]:
def balance(df, target_col, samples, class_b, num_b):
    d, m = divmod(samples, num_b)
    for _ in range(d):
        tmp = df.loc[df[target_col] == class_b].sample(n = num_b)
        df = pd.concat([df, tmp], ignore_index=True)
    if m:
        tmp = df.loc[df[target_col] == class_b].sample(n = m)
        df = pd.concat([df, tmp], ignore_index=True)
    return df

In [33]:
def balance_dataset(df, target_col):
    (class_a, num_a), (class_b, num_b) = sorted(dict(df[target_col].value_counts()).items())
    
    samples = abs(num_a - num_b)
    if (num_a > num_b):
        df = balance(df, target_col, samples, class_b, num_b)
    else:
        balance(df, target_col, samples, class_a, num_a)
        
    return df

In [14]:
def clean_df(dfy: pd.DataFrame, text_col:str):
    lst = []
    drop_idx = []
    for i, line in dfy.iterrows():
        try:
            lst += [cleaning_text(line[text_col])]
        except Exception:
            drop_idx += [i]
    dfy = dfy.drop(drop_idx)
    tmp = pd.DataFrame({text_col: lst}, index=dfy[text_col].index)    
    dfy.update(tmp)
    return dfy

In [48]:
# def make_preds(dfy, col:str, src_df:pd.DataFrame):
#     change_data = dfy.loc[dfy[col].isna() , 'text']
#     print('change_data_shape', change_data.shape)
#     train = dfy.copy()
#     print(train.shape)
#     print(train[col].value_counts())
#     print(train[col].isna().sum())
#     train[col] = train[col].apply(filt)
#     print(train.shape)
#     train = train.drop(train[train[col] == -1].index)
#     train = balance_dataset(train, col) 
#     print(train.text)

#     train[col] = train[col].astype(int)
#     train = drop_by_col(train, col='text')

#     print(train.shape)

#     df_train, df_test = train_test_split(train, 
#                                         random_state=42, 
#                                         test_size=0.1, 
#                                         stratify=train[col]
#                                         )

#     train_corpus = df_train.text.values
#     test_corpus = df_test.text.values

#     y_train = df_train[col]
#     y_test = df_test[col]

#     print("vectorizing")

#     vectorizer = TfidfVectorizer(ngram_range=(2,4), analyzer='char_wb', max_df=0.8, min_df=10)
#     vectorizer.fit(train_corpus)
#     X_train = vectorizer.transform(train_corpus)
#     X_test = vectorizer.transform(test_corpus)
#     yt_change = vectorizer.transform(change_data.values)

#     rfc = RandomForestClassifier(random_state=0)
#     rfc.fit(X_train, y_train)

#     print("predict")
#     y_pred = rfc.predict(X_test)
#     print(classification_report(y_test, y_pred))
#     yt_pred = rfc.predict(yt_change)

#     print('changing')
#     tmp = change_data.index
#     s = pd.DataFrame(data={col : yt_pred}, index=tmp)
#     src_df.update(s)

In [92]:
# def make_preds(dfy, col:str, src_df:pd.DataFrame, train_df:pd.DataFrame|None=None):
#     change_data = dfy.loc[dfy[col].isna() , 'text']
#     if len(change_data) == 0:
#         return
#     print('change_data_shape', change_data.shape)

#     if train_df is None:
#         train = dfy.copy()
#         print(train.shape)
#     else:
#         train = train_df
        
#     print(train[col].value_counts())
#     print(train[col].isna().sum())
#     train[col] = train[col].apply(filt)
#     print(train.shape)
#     train = train.drop(train[train[col] == -1].index)
#     train = balance_dataset(train, col) 
#     print(train.text)

#     train[col] = train[col].astype(int)
#     train = drop_by_col(train, col='text')

#     print(train.shape)

    

#     df_train, df_test = train_test_split(train, 
#                                         random_state=42, 
#                                         test_size=0.1, 
#                                         stratify=train[col]
#                                         )

#     train_corpus = df_train.text.values
#     test_corpus = df_test.text.values

#     y_train = df_train[col]
#     y_test = df_test[col]

#     print("vectorizing")

#     vectorizer = TfidfVectorizer(ngram_range=(2,4), analyzer='char_wb', max_df=0.8, min_df=10)
#     vectorizer.fit(train_corpus)
#     X_train = vectorizer.transform(train_corpus)
#     X_test = vectorizer.transform(test_corpus)
#     yt_change = vectorizer.transform(change_data.values)

#     rfc = RandomForestClassifier(random_state=0)
#     rfc.fit(X_train, y_train)

#     print("predict")
#     y_pred = rfc.predict(X_test)
#     print(classification_report(y_test, y_pred))
#     yt_pred = rfc.predict(yt_change)

#     print('changing')
#     tmp = change_data.index
#     s = pd.DataFrame(data={col : yt_pred}, index=tmp)
#     src_df.update(s)

In [119]:
def gs_rfc(X_train, y_train, scoring=f1_score):
    rfc = RandomForestClassifier(random_state=0)

    param_grid = { 
        'n_estimators': [50, 100, 200, 500],
        'max_features': ['sqrt', 'log2'],
        'max_depth' : [4,5,6,7,8],
        'criterion' :['gini', 'entropy']
    }

    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, scoring=scoring)
    CV_rfc.fit(X_train, y_train)
    return CV_rfc

def gs_xgboost(X_train, y_train, scoring=f1_score):
    xgb_model = xgb.XGBClassifier()
    parameters = {'nthread':[4], 
              'objective':['binary:logistic'],
              'learning_rate': [0.05], 
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], 
              'missing':[-1],
              'seed': [0]}
    clf = GridSearchCV(xgb_model, parameters, n_jobs=-1, 
                    cv=5, 
                    scoring=scoring,
                    verbose=0, refit=True)
    clf.fit(X_train, y_train)
    return clf

def gs_logreg(X_train, y_train, scoring=f1_score):
    logreg = LogisticRegression(random_state=0)
    parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
                {'penalty':['l2']},
                {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]
    grid_search = GridSearchCV(estimator = logreg,  
                            param_grid = parameters,
                            scoring = scoring,
                            cv = 5,
                            verbose=0)
    grid_search.fit(X_train, y_train)
    return grid_search

def gs_svc(X_train, y_train, scoring=f1_score):
    classifier = SVC(random_state=0)
    x = [1.0,10.0,100.0,500.0,1000.0]
    y = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    z = [2,3,4]    
    parameters=[{'C': x,'kernel': ['linear']},
                {'C': x,'kernel': ['rbf'],'gamma': y} ,
                {'C': x,'kernel': ['poly'],'gamma': y,'degree': z}
            ]
    gridsearch=GridSearchCV(estimator = classifier,
                            param_grid = parameters,
                            scoring=scoring,
                            cv=10,
                            n_jobs=-1)
    gridsearch=gridsearch.fit(X_train,y_train)

def choose_estimator(X_test, y_test, estimators, scoring=f1_score, strategy='max'):
    maxx = 0
    best_estimator = None

    for est in estimators:
        cross_val = cross_val_score(est, X_test, y_test, scoring=scoring, cv=3).mean()
        if strategy == 'max':
            if cross_val > maxx:
                best_estimator = est
                maxx = cross_val
    model_name = type(best_estimator).__name__
    print(model_name)
    print(best_estimator.best_params_)
    print('Best score:', maxx)
    return best_estimator

In [106]:
def make_preds(dfy, col:str, src_df:pd.DataFrame, train_df:pd.DataFrame|None=None, multi_estim:bool=False):
    change_data = dfy.loc[dfy[col].isna() , 'text']
    if len(change_data) == 0:
        return
    print('change_data_shape', change_data.shape)

    if train_df is None:
        train = dfy.copy()
        print(train.shape)
    else:
        train = train_df
        
    print(train[col].value_counts())
    print(train[col].isna().sum())
    train[col] = train[col].apply(filt)
    print(train.shape)
    train = train.drop(train[train[col] == -1].index)
    train = balance_dataset(train, col) 
    print(train.text)

    train[col] = train[col].astype(int)
    train = drop_by_col(train, col='text')

    print(train.shape)

    

    df_train, df_test = train_test_split(train, 
                                        random_state=42, 
                                        test_size=0.1, 
                                        stratify=train[col]
                                        )

    train_corpus = df_train.text.values
    test_corpus = df_test.text.values

    y_train = df_train[col]
    y_test = df_test[col]

    print("vectorizing")

    vectorizer = TfidfVectorizer(ngram_range=(2,4), analyzer='char_wb', max_df=0.8, min_df=10)
    vectorizer.fit(train_corpus)
    X_train = vectorizer.transform(train_corpus)
    X_test = vectorizer.transform(test_corpus)
    yt_change = vectorizer.transform(change_data.values)

    est = RandomForestClassifier(random_state=0)

    if multi_estim:

        print('\nfit\n')
        rfc = gs_rfc(X_train, y_train)
        xgboost = gs_xgboost(X_train, y_train)
        logreg = gs_logreg(X_train, y_train)
        svc = gs_svc(X_train, y_train)

        est = choose_estimator(X_test, y_test, (rfc, xgboost, logreg, svc))
    else:
        est.fit(X_train, y_train)    
        
    print("\npredict\n")
    y_pred = est.predict(X_test)
    print(classification_report(y_test, y_pred))
    yt_pred = est.predict(yt_change)

    print('changing')
    tmp = change_data.index
    s = pd.DataFrame(data={col : yt_pred}, index=tmp)
    src_df.update(s)

In [69]:
def columns_differ(steam_df, youtube_df, display=True):
        cols_steam     = set(steam_df.columns)
        cols_youtube = set(youtube_df.columns)

        common_cols = cols_steam.intersection(cols_youtube)
        different_cols_f = cols_steam.difference(cols_youtube)
        different_cols_s = cols_youtube.difference(cols_steam)

        if display:
            print("Common Columns")
            print(*common_cols, sep='\n')

            print("\nDifferent Columns")
            print('First:')
            print(*different_cols_f, sep='\n')
            print('Second:')
            print(*different_cols_s, sep='\n')
        return different_cols_f, different_cols_s

## YouTube

In [16]:
src_yt_df = pd.read_csv('yt_par.csv')
dfy = src_yt_df.copy()
dfy = dfy.rename(columns={'Комментарий (текст: скопировать оригинал)':'text'})
cat_cols_yt = ['До 30 (1) или 30+ (0)?', 'Политический ли комментарий? (1/0)', 'Есть ли в видео общест.-полит. оценка? (1/0)',
            'Война и мир', 'Игра в войну ', 'Реальность виртуального ',
       'Антимилитаризм', 'Свой-чужой ', 'Патриотизм ',
       'Положительная оценка СССР', 'Отрицательная оценка СССР',
       'Критика 90-ых годов / Все о 90-ых', 'Клюква', 'Политическая ирония',
       'Образ будущего ', 'Повесточка',
       'Запрос на социальную справедливость и равенство',
       'Запрос на объективность ', 'Запрос на альтернативность ',
       'Запрос на субъектность ', 'Негативный национализм',
       'Позитивный национализм ', 'Поддержка СВО', 'Критика СВО',
       'СВО и ее последствия', 'Актуальные проблемы ', 'Критика власти/режима',
       'Поддержка власти/режима', 'Принадлежность к идеологии: левый',
       'Принадлежность к идеологии: правый',
       'Принадлежность к идеологии: либерал/центрист',
       'Ценности: гуманизм/антигуманизм',
       'Мобилизационный и/или мотивационный потенциал ',
       'Международная и/или мировая политика', 'Оппозиция']

preprocess whole data

In [ ]:
dfy = drop_by_col(dfy, col='text')
dfy = clean_df(dfy, 'text')
dfy.to_csv("yt_clean_preproc_full.csv")

In [46]:
df = pd.read_csv("yt_clean_preproc_full.csv", index_col=0)
df = drop_by_col(df, 'text')

In [47]:
df.shape

(45559, 68)

In [49]:
for col in cat_cols_yt:
    make_preds(df, col, src_yt_df)

change_data_shape (44740,)
(45559, 68)
До 30 (1) или 30+ (0)?
0.0    620
1.0    199
Name: count, dtype: int64
44740
(45559, 68)
0       помесь детройт киберпанк всё это родный душевн...
1       тысяча восемьсот семнадцать лежать свой кварти...
2                     игра класс ак хотеться вернуть ссср
3                              любить игра это мечта ссср
4                     бояться скорый время всё также игра
                              ...                        
1235    эксперт почему считать глупый это просто кадро...
1236    уверенный еслиб разраб остаться россия тупо за...
1237    нагло лгать весь запад большой поставлять наём...
1238    тысяча семьсот тридцать восемь скорее весь это...
1239    самый главное это сериал разница национальност...
Name: text, Length: 1240, dtype: object
(1240, 68)
vectorizing
predict
              precision    recall  f1-score   support

           0       0.95      0.90      0.93        62
           1       0.91      0.95      0.93        62


In [50]:
src_yt_df.to_csv("updated_parser_yt_info.csv")

Категории, на которых F1-score < 0.98:
1. До 30 (1) или 30+ (0)? (0.93)
2. Политический ли комментарий? (1/0) (0 - 0.56, 1 - 0.83)
3. Есть ли в видео общест.-полит. оценка? (1/0) (0 - 0.68, 1 - 0.79)
4. Война и мир (0.96)

Попробуем улучшить качество в дальнейшем.

Можно посмотреть пропущенные значения.

In [55]:
for col in src_yt_df.columns:
    print(src_yt_df[col].isna().sum(), col, )

32523 №
397 Название видео
45284 Кто делал анализ? Фамилия, пароли, явки!
45357 Ссылка Youtube
45357 Дата
45357 Просмотры
45358 Лайки
45357 Комментарии
45357 Всего: П+Л+К
45357 % П от ПЛК
45357 % Л от ПЛК
45357 % К от ПЛК
45343 Категория (Фильмы, Игры, Музыка, Оппозиционеры)
45344 Временной интервал (с 01.01.23 по 31.12.23): указывать интервал
45343 Тема, к которой относится видео, что было в Google-трендах (перечисление; поиск по фильтрам)
610 Кол-во ответов на комментарий
610 id канала
3 Комментарий (текст: скопировать оригинал)
44752 Возраст автора (интервалом): До 17 лет; 17-22; 23 и старше; 30+; любой
11 До 30 (1) или 30+ (0)?
44747 Аргументы от ChatGPT
612 Лайки комментария
36617 ТИП политического комментария (тема)
7 Политический ли комментарий? (1/0)
8 Есть ли в видео общест.-полит. оценка? (1/0)
35970 Тэги по видео (по описанию: ключ. слова, только ПОЛИТ СОСТАВЛ)
6 Война и мир
6 Игра в войну 
6 Реальность виртуального 
6 Антимилитаризм
6 Свой-чужой 
6 Патриотизм 
6 Положительн

## STEAM

In [66]:
src_st_df = pd.read_csv('st_par.csv')
dfs = src_st_df.copy()
dfs = dfs.rename(columns={'Обзор (текст: скопировать оригинал): "reviews" (сами отзывы)':'text'})
cat_cols_st = ['До 30 (1) или 30+ (0)?', 'Политический ли комментарий? (1/0)',
       'Есть ли в игре общест.-полит. оценка? (1/0)', 'Война и мир',
       'Игра в войну ', 'Реальность виртуального ', 'Антимилитаризм',
       'Свой-чужой ', 'Патриотизм ', 'Положительная оценка СССР',
       'Отрицательная оценка СССР', 'Критика 90-ых годов / Все о 90-ых',
       'Клюква', 'Политическая ирония', 'Образ будущего ', 'Повесточка',
       'Запрос на социальную справедливость и равенство',
       'Запрос на объективность ', 'Запрос на альтернативность ',
       'Запрос на субъектность ', 'Негативный национализм',
       'Позитивный национализм ', 'Поддержка СВО', 'Критика СВО',
       'СВО и ее последствия', 'Актуальные проблемы ', 'Критика власти/режима',
       'Поддержка власти/режима', 'Принадлежность к идеологии: левый',
       'Принадлежность к идеологии: правый',
       'Принадлежность к идеологии: либерал/центрист',
       'Ценности: гуманизм/антигуманизм',
       'Мобилизационный и/или мотивационный потенциал ',
       'Международная и/или мировая политика', 'Оппозиция']

In [57]:
dfs = drop_by_col(dfs, col='text')
dfs = clean_df(dfs, 'text')
dfs.to_csv("steam_clean_preproc_full.csv")

In [63]:
df.columns

Index(['№', 'Название игры', 'Кто делал анализ? Фамилия!',
       'Ссылка на Steam/VK Play', 'Ссылка на игру в VK', 'Жанр ',
       '"review_score" (суммарная оценка игры от всех отзывов от 1 до 10)',
       '"review_score_desc" (суммарная оценка исходя от "Очень положительные" или "Смешанные")',
       '"total_positive" (количество положительных отзывов об игре на русском языке)',
       '"total_negative" (количество негативных отзывов об игре на русском языке)',
       '"total_reviews" (суммарное количество отзывов об игре на русском языке)',
       'Оценка / Процент', 'Дата обработки',
       '"recommendationid" (уникальный идентификатор отзыва)', 'text',
       'Возраст автора (интервалом): До 17 лет; 17-22; 23 и старше; 30+; любой',
       'До 30 (1) или 30+ (0)?', 'Аргументы от ChatGPT',
       'ТИП политического комментария (тема)',
       'Политический ли комментарий? (1/0)',
       'Есть ли в игре общест.-полит. оценка? (1/0)',
       'Тэги по игре (по описанию: ключ. слова)',

In [67]:
for col in cat_cols_st:
    print(df[col].value_counts())

До 30 (1) или 30+ (0)?
0.0    850
1.0    438
Name: count, dtype: int64
Политический ли комментарий? (1/0)
1      1297
0       888
А         1
укр       1
Name: count, dtype: int64
Есть ли в игре общест.-полит. оценка? (1/0)
1.0    1244
0.0     942
Name: count, dtype: int64
Война и мир
0.0    1589
1.0     593
Name: count, dtype: int64
Игра в войну 
0.0    1900
1.0     282
Name: count, dtype: int64
Реальность виртуального 
0    1743
1     438
й       1
Name: count, dtype: int64
Антимилитаризм
0.0    1997
1.0     185
Name: count, dtype: int64
Свой-чужой 
0    1955
1     226
й       1
Name: count, dtype: int64
Патриотизм 
0.0    1995
1.0     187
Name: count, dtype: int64
Положительная оценка СССР
0.0    2125
1.0      57
Name: count, dtype: int64
Отрицательная оценка СССР
0.0    2159
1.0      23
Name: count, dtype: int64
Критика 90-ых годов / Все о 90-ых
0.0    2167
1.0      15
Name: count, dtype: int64
Клюква
0.0    2082
1.0     100
Name: count, dtype: int64
Политическая ирония
0    1918
1

In [58]:
df = drop_by_col(df, 'text')
df.shape

(2187, 79)

Поскольку для Steam данных мало, используем датасет YouTube для обучения моделей.

In [73]:
df1 = pd.read_csv("yt_clean_preproc_full.csv", index_col=0)
df2 = pd.read_csv("steam_clean_preproc_full.csv", index_col=0)
df1 = df1.rename(columns={'Политический ли комментарий? (1/0)': 'label', 'Есть ли в видео общест.-полит. оценка? (1/0)': 'assessment'})
df2 = df2.rename(columns={'Политический ли комментарий? (1/0)': 'label', 'Есть ли в игре общест.-полит. оценка? (1/0)': 'assessment'})
names = columns_differ(df1, df2)

Common Columns
Личности и\или политики
Мобилизационный и/или мотивационный потенциал 
Критика власти/режима
Запрос на альтернативность 
Актуальные проблемы 
Международная и/или мировая политика
Политическая ирония
Позитивный национализм 
Критика СВО
Критика 90-ых годов / Все о 90-ых
Оппозиция
Ценности: гуманизм/антигуманизм
Образ народа
Клюква
Поддержка СВО
Образ будущего 
label
Запрос на субъектность 
Положительная оценка СССР
Свой-чужой 
Примечания
Поддержка власти/режима
Кто пишет отзыв? (примерный возрастной интервал)
Иное
text
Запрос на объективность 
Игра в войну 
ТЭГИ ИНОГО (Темы, которые не были выделены отдельно!)
До 30 (1) или 30+ (0)?
Антимилитаризм
Негативный национализм
assessment
СВО и ее последствия
ТИП политического комментария (тема)
Аргументы от ChatGPT
Патриотизм 
Кол-во ТИПОВ
Принадлежность к идеологии: левый
Принадлежность к идеологии: либерал/центрист
№
Запрос на социальную справедливость и равенство
Война и мир
Реальность виртуального 
Отрицательная оценка СССР
П

Обновим названия колонок и в итоговом датасете. Их можно будет изменить потом.

In [88]:
src_yt_df = src_yt_df.rename(columns={'Политический ли комментарий? (1/0)': 'label', 'Есть ли в видео общест.-полит. оценка? (1/0)': 'assessment'})
src_st_df = src_st_df.rename(columns={'Политический ли комментарий? (1/0)': 'label', 'Есть ли в игре общест.-полит. оценка? (1/0)': 'assessment'})

In [75]:
a, b = names
df1 = df1.drop(columns=a)
df2 = df2.drop(columns=b)

In [76]:
df1.shape, df2.shape

((45567, 47), (2193, 47))

In [77]:
df = pd.concat([df1, df2], ignore_index=True)

In [84]:
print(df1.text.isna().sum(),
    df2.text.isna().sum(),
    df.text.isna().sum())

8 6 14


In [85]:
df1 = drop_by_col(df1, 'text')
df2 = drop_by_col(df2, 'text')
df = drop_by_col(df, 'text')

In [86]:
print(df1.text.isna().sum(),
    df2.text.isna().sum(),
    df.text.isna().sum())

0 0 0


In [90]:
cat_cols_st = ['До 30 (1) или 30+ (0)?', 'label',
       'assessment', 'Война и мир',
       'Игра в войну ', 'Реальность виртуального ', 'Антимилитаризм',
       'Свой-чужой ', 'Патриотизм ', 'Положительная оценка СССР',
       'Отрицательная оценка СССР', 'Критика 90-ых годов / Все о 90-ых',
       'Клюква', 'Политическая ирония', 'Образ будущего ', 'Повесточка',
       'Запрос на социальную справедливость и равенство',
       'Запрос на объективность ', 'Запрос на альтернативность ',
       'Запрос на субъектность ', 'Негативный национализм',
       'Позитивный национализм ', 'Поддержка СВО', 'Критика СВО',
       'СВО и ее последствия', 'Актуальные проблемы ', 'Критика власти/режима',
       'Поддержка власти/режима', 'Принадлежность к идеологии: левый',
       'Принадлежность к идеологии: правый',
       'Принадлежность к идеологии: либерал/центрист',
       'Ценности: гуманизм/антигуманизм',
       'Мобилизационный и/или мотивационный потенциал ',
       'Международная и/или мировая политика', 'Оппозиция']
cat_cols_yt = cat_cols_st[:]

In [93]:
for col in cat_cols_st:
    make_preds(df2, col, src_st_df, df)

change_data_shape (899,)
До 30 (1) или 30+ (0)?
-1    45639
 0     1470
 1      637
Name: count, dtype: int64
0
(47746, 47)
0       помесь детройт киберпанк всё это родный душевн...
1       тысяча восемьсот семнадцать лежать свой кварти...
2                     игра класс ак хотеться вернуть ссср
3                              любить игра это мечта ссср
4                     бояться скорый время всё также игра
                              ...                        
2935    наш белорусский научный картошка хороший мир м...
2936    давно знать игра двор приходить школа запускат...
2937    процитировать ипполит гадость гадость это ваш ...
2938    поздравлять действительно обречь жить прокляты...
2939    показать видео социология видно херсонский обл...
Name: text, Length: 2940, dtype: object
(2940, 47)
vectorizing
predict
              precision    recall  f1-score   support

           0       0.99      0.84      0.91       147
           1       0.86      0.99      0.92       147

   

F1-score < 0.98 для колонок:
1. До 30 (1) или 30+ (0)? (0 - 0.91, 1 - 0.92)
2. assessment (0 - 0.61, 1 - 0.75)
3. Война и мир (0 - 0.95, 1 - 0.96)

In [99]:
src_st_df.to_csv("updated_parser_st_info.csv")

## Улучшение предсказаний

### Steam Data

In [120]:
# X_train = np.array([1, 1, 1,0,0,0, 0]).reshape(-1, 1)
# y_train = np.array([0, 0, 0, 0,0,0,0]).reshape(-1, 1)
# X_test = np.array([1, 1, 1, 0,0,0,0]).reshape(-1, 1)
# y_test = np.array([0, 0, 0, 1,0,0,0]).reshape(-1, 1)

# print('\nfit\n')
# rfc = gs_rfc(X_train, y_train)
# xgboost = gs_xgboost(X_train, y_train)
# logreg = gs_logreg(X_train, y_train)
# svc = gs_svc(X_train, y_train)

# est = choose_estimator(X_test, y_test, (rfc, xgboost, logreg, svc))



fit



ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\zlata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\zlata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zlata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1301, in fit
    raise ValueError(
ValueError: This solver needs samples of at least 2 classes in the da

In [121]:
for col in ['До 30 (1) или 30+ (0)?', 'assessment', 'Война и мир']:
    make_preds(df2, col, src_st_df, df, multi_estim=True)

change_data_shape (899,)
До 30 (1) или 30+ (0)?
-1    45639
 0     1470
 1      637
Name: count, dtype: int64
0
(47746, 47)
0       помесь детройт киберпанк всё это родный душевн...
1       тысяча восемьсот семнадцать лежать свой кварти...
2                     игра класс ак хотеться вернуть ссср
3                              любить игра это мечта ссср
4                     бояться скорый время всё также игра
                              ...                        
2935    зайти карта бахмат сразу черва черва черва чер...
2936    приобрести игра сразу начать смотреть ролик иг...
2937    респект пригожий заварить каша нужный страна ч...
2938    бойня кровавый невозможно выживать либо команд...
2939    игра интересный плюс мочь выделить интересный ...
Name: text, Length: 2940, dtype: object
(2940, 47)
vectorizing

fit



ValueError: scoring value <function f1_score at 0x000001B039DD0360> looks like it is a metric function rather than a scorer. A scorer should require an estimator as its first parameter. Please use `make_scorer` to convert a metric to a scorer.

### YouTube Data

### Use BERT

In [ ]:
from safetensors import safe_open
tensors = {}
with safe_open(r"C:\Users\zlata\Documents\parser_ign\models\my_model_v2\model.safetensors", framework="pt", device='cpu') as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

model.load_state_dict(tensors)
model.eval()

train_data = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train}))
test_data = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test}))

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))

train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels)

labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)